In [1]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
from sklearn.metrics import classification_report

# Load the Haar cascade file for face detection
face_cascade = cv2.CascadeClassifier(r'D:\MTECH\PROJECT\DM\SECRET\CASCADE\haarcascade_frontalface_default.xml')

# List of emotions
emotions = ['anger', 'contempt', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Number of orthogonal vectors to keep
num_vectors = 40

# List to store feature vectors and labels
feature_vectors = []
labels = []

# Path to the dataset directories
dataset_dirs = [
    r'D:\MTECH\PROJECT\CV\FINAL - ORIGINAL\DATASET\AffectNet\data_relabeled_balanced_1x\train',
    r'D:\MTECH\PROJECT\CV\FINAL - ORIGINAL\DATASET\AffectNet\data_relabeled_balanced_2x\train',
    r'D:\MTECH\PROJECT\CV\FINAL - ORIGINAL\DATASET\AffectNet\data_relabeled_balanced_3x\train'
]

# Iterate over the dataset directories
for dataset_dir in dataset_dirs:
    # Iterate over the emotions
    for emotion in emotions:
        emotion_dir = os.path.join(dataset_dir, emotion)
        image_files = os.listdir(emotion_dir)

        for image_file in image_files:
            image_path = os.path.join(emotion_dir, image_file)

            # Load the input image
            image = cv2.imread(image_path)

            # Convert the image to grayscale
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Detect faces in the grayscale image
            faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.5, minNeighbors=5, minSize=(30, 30))

            # Print the number of detected faces
            print("Number of detected faces:", len(faces))

            # Iterate over the detected faces
            for (x, y, w, h) in faces:
                # Extract the face region
                face = gray_image[y:y + h, x:x + w]

                # Resize the face image to 50x50 pixels
                resized_face = cv2.resize(face, (50, 50), interpolation=cv2.INTER_LINEAR)

                # Calculate LBP (Local Binary Patterns) values
                lbp = local_binary_pattern(resized_face, 8, 1, method='uniform')

                # Calculate the histogram of LBP values
                histogram, _ = np.histogram(lbp.ravel(), bins=256, range=(0, 256))

                # Append the histogram and label to the feature vectors and labels lists
                feature_vectors.append(histogram)
                labels.append(emotion)

# Convert feature vectors and labels to NumPy arrays
feature_vectors = np.array(feature_vectors)
labels = np.array(labels)

# Print the number of feature vectors and labels
print("Number of feature vectors:", len(feature_vectors))
print("Number of labels:", len(labels))

# Split the dataset into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(
    feature_vectors, labels, test_size=0.2, random_state=42
)

# Normalize the feature vectors
train_features = train_features / 255.0
test_features = test_features / 255.0

# Reshape the feature vectors for CNN input
train_features = train_features.reshape(train_features.shape[0], 50, 50, 1)
test_features = test_features.reshape(test_features.shape[0], 50, 50, 1)

# Define the CNN model architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(emotions), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_features, train_labels, epochs=10, batch_size=32)

# Evaluate the model on the testing set
test_loss, test_accuracy = model.evaluate(test_features, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Save the trained model
model.save('emotion_model.h5')
